In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

import pylab as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm
from statsmodels.formula.api import ols

from statistics import mean

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score as r2

from sklearn.model_selection import train_test_split as tts

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoLars
from sklearn.linear_model import BayesianRidge

from tqdm import tqdm
import math

In [2]:
def analisis(errores, cota = 0.25, paises = 10):
    salaries_data = pd.read_csv('salaries_data.csv')
    dimension_original_salaries_data = salaries_data.shape
    test = pd.read_csv('test.csv')
    dimension_original_test = test.shape
    
    iqr = salaries_data.salary_in_usd.quantile(1-cota) - salaries_data.salary_in_usd.quantile(cota)
    cutoff = iqr * 1.5
    lower =  salaries_data.salary_in_usd.quantile(cota) - cutoff
    upper = salaries_data.salary_in_usd.quantile(1-cota) + cutoff
    salaries_data.drop(salaries_data[(salaries_data.salary_in_usd < lower) | (salaries_data.salary_in_usd > upper)].index, axis=0, inplace = True)
    
    X_salaries_data = salaries_data.drop('salary_in_usd', axis=1)
    y_salaries_data = salaries_data.salary_in_usd
    
    data = pd.concat([X_salaries_data,test])
    dimension_original_data = data.shape
    
    data.work_year = data.work_year.apply(lambda x: str(x))
    data.experience_level.replace({'EN': 0, 'MI': 1, 'SE': 2, 'EX': 3}, inplace=True)
    
    new = []
    for server in data.job_title:
        if 'scien' in server.lower():
            new.append('Data Scientist')
        elif 'engine' in server.lower():
            new.append('Data Engineer')
        elif 'analy' in server.lower():
            new.append('Data Analyst')
        elif 'archi' in server.lower():
            new.append('Data Architect')
        elif 'machi' in server.lower():
            new.append('Machine Learning')
        else: new.append('Other')
    data.job_title = new
    
    data = data.drop(['employee_residence'], axis = 1)
    
    data.remote_ratio.replace({0: 'CERO', 50: 'HALF', 100: 'FULL'}, inplace=True)
    
    top_paises = pd.DataFrame(data.company_location.value_counts())[:paises]
    
    data.company_location = ['OTHER' if c not in top_paises.index else c for c in data.company_location]
    
    salario_medio_pais = {'US': 3600, 'FR': 3201,'GR': 1407, 'LU': 3400, 'SI': 1990,
                        'DE': 3393, 'IN': 1454, 'GB': 2997, 'PK': 747, 'MD': 1162,
                        'JP': 2808, 'CA': 3082, 'AS': 2282, 'IE': 3016, 'AE': 4302,
                        'MX': 1123, 'VN': 1388, 'BE': 2779, 'KE': 986, 'ES': 2280,
                        'CH': 5000, 'CL': 1085, 'CN': 1819, 'DK': 5550, 'TR': 1371,
                        'NZ': 3050, 'PL': 1441, 'UA': 1710, 'AU': 3599, 'NG': 455,
                        'EE': 2299, 'CZ': 2282, 'AT': 2777, 'BR': 800, 'DZ': 905,
                        'IR': 1768, 'NL': 3458, 'HU': 1876, 'PT': 1461, 'HN': 1134,
                        'MT': 1914, 'RO': 1751, 'SG': 6550, 'IT': 2143, 'HR': 1827,
                        'IQ': 1324, 'IL': 2528, 'RU': 1790, 'CO': 861, 'MY': 2195}

    salario_medio_pais['OTHER'] = int(round(mean([salario_medio_pais[i] for i in top_paises.index]),0))

    new = []
    for i, e  in enumerate(data.work_year):
        new.append(salario_medio_pais[data.company_location.iloc[i]])
    data['salario_medio_pais'] = new

    data = pd.get_dummies(data)

    X_salaries_data = data[:len(X_salaries_data)]
    test = data[len(X_salaries_data):]
    
    salaries_data = pd.concat([X_salaries_data,y_salaries_data], axis = 1)

    X_train, X_test,y_train, y_test = tts(X_salaries_data, y_salaries_data, train_size=0.8, test_size=0.2, random_state=22)
    
    modelos = [LogisticRegression(max_iter=2000)
           , LinearRegression()
           , Ridge(alpha=.5)
           , RidgeCV(alphas=np.logspace(-6, 6, 13))
           , Lasso(alpha=0.1)
           , LassoLars(alpha=.1, normalize=False)
           , BayesianRidge()
          ]
    
    for i in modelos:
        reg = i
        reg.fit(X_train, y_train)
        y_pred=reg.predict(X_test)
        #y_prob=reg.predict_proba(X_test)

        accuracy = sum(y_pred==y_test)/y_pred.shape[0] * 100

        errores = errores.append({'Modelo': i
                                  , 'Cota': cota
                                  , 'Num Paises': paises
                                  , 'Acierto exacto (%)': round(accuracy, 2)
                                  , 'RMSE': mse(y_test, y_pred, squared=False)
                                  , 'MSE': mse(y_test, y_pred)
                                  , 'MAE': mae(y_test, y_pred)
                                  , 'R2': r2(y_test, y_pred)
                                 }, ignore_index=True)

    return errores, mse(y_test, y_pred, squared=False), r2(y_test, y_pred)
    

In [3]:
%%time

errores = pd.DataFrame(columns = ['Modelo', 'Cota', 'Num Paises', 'Acierto exacto (%)', 'RMSE', 'MSE', 'MAE', 'R2'])
top = [float(math.inf)]
for i in tqdm(range(50)):
    cota = i/100 
    for num_paises in range(1,50):
        errores, rmse_val, r2_val = analisis(errores, cota, num_paises)
        print(cota, '\t',  num_paises,'\t', rmse_val,'\t', r2_val)

  0%|                                                    | 0/50 [00:00<?, ?it/s]

0.0 	 1 	 37817.1142229313 	 0.5507901966134011
0.0 	 2 	 47953.29379917728 	 0.27771375998107495
0.0 	 3 	 48333.57004196673 	 0.2662126782203419
0.0 	 4 	 49629.00902552653 	 0.22635154626672882
0.0 	 5 	 49621.369249962154 	 0.22658971526280036
0.0 	 6 	 49840.019348041875 	 0.2197588353628802
0.0 	 7 	 49480.34401884087 	 0.2309795729143338
0.0 	 8 	 50223.89409963394 	 0.20769349751565858
0.0 	 9 	 50103.548896417975 	 0.21148595729224362
0.0 	 10 	 49967.560445599316 	 0.21576043636809294
0.0 	 11 	 49993.89462981661 	 0.21493358986596633
0.0 	 12 	 49694.19348360163 	 0.2243179383286834
0.0 	 13 	 49632.87656451361 	 0.22623096226950046
0.0 	 14 	 49440.02833256236 	 0.23223223015894146
0.0 	 15 	 52058.39179281483 	 0.14875622274784495
0.0 	 16 	 52053.4927242708 	 0.14891643151500655
0.0 	 17 	 51734.76890789968 	 0.15930690192451502
0.0 	 18 	 51747.35199888229 	 0.15889790022469796
0.0 	 19 	 51184.9485163236 	 0.17708117642138244
0.0 	 20 	 51057.71401490626 	 0.18116728118

  2%|▊                                        | 1/50 [04:31<3:41:56, 271.76s/it]

0.0 	 49 	 49790.988845669475 	 0.22129321675540703
0.01 	 1 	 37817.1142229313 	 0.5507901966134011
0.01 	 2 	 47953.29379917728 	 0.27771375998107495
0.01 	 3 	 48333.57004196673 	 0.2662126782203419
0.01 	 4 	 49629.00902552653 	 0.22635154626672882
0.01 	 5 	 49621.369249962154 	 0.22658971526280036
0.01 	 6 	 49840.019348041875 	 0.2197588353628802
0.01 	 7 	 49480.34401884087 	 0.2309795729143338
0.01 	 8 	 50223.89409963394 	 0.20769349751565858
0.01 	 9 	 50103.548896417975 	 0.21148595729224362
0.01 	 10 	 49967.560445599316 	 0.21576043636809294
0.01 	 11 	 49993.89462981661 	 0.21493358986596633
0.01 	 12 	 49694.19348360163 	 0.2243179383286834
0.01 	 13 	 49632.87656451361 	 0.22623096226950046
0.01 	 14 	 49440.02833256236 	 0.23223223015894146
0.01 	 15 	 52058.39179281483 	 0.14875622274784495
0.01 	 16 	 52053.4927242708 	 0.14891643151500655
0.01 	 17 	 51734.76890789968 	 0.15930690192451502
0.01 	 18 	 51747.35199888229 	 0.15889790022469796
0.01 	 19 	 51184.948516

  4%|█▌                                      | 2/50 [25:20<11:17:16, 846.60s/it]

...

0.47 	 49 	 8885.778763914504 	 -0.0014923896424710037
0.48 	 1 	 3011.337799899659 	 -0.24326300735764983
0.48 	 2 	 3011.3062495439526 	 -0.24323695569075654
0.48 	 3 	 3011.259919676551 	 -0.2431987008236367
0.48 	 4 	 3011.290488706284 	 -0.24322394180047713
0.48 	 5 	 2954.885710165477 	 -0.19708623190834862
0.48 	 6 	 2958.003109495494 	 -0.1996134121126827
0.48 	 7 	 2952.1720181080714 	 -0.1948884943053959
0.48 	 8 	 2946.0009851541886 	 -0.18989827728814657
0.48 	 9 	 2943.7868053650977 	 -0.18811032226411273
0.48 	 10 	 2957.67182567006 	 -0.19934472391240976
0.48 	 11 	 2948.4187607500076 	 -0.19185217178174163
0.48 	 12 	 2948.509686883181 	 -0.1919256838498058
0.48 	 13 	 2957.541274501058 	 -0.19923884847432505
0.48 	 14 	 2957.8068690467626 	 -0.1994542473946721
0.48 	 15 	 2953.589000974333 	 -0.19603581421004956
0.48 	 16 	 2952.60038898403 	 -0.19523528477958152
0.48 	 17 	 2949.6997747238224 	 -0.19288805649442153
0.48 	 18 	 2948.9987486323184 	 -0.1923211199621837


 98%|████████████████████████████████████████▏| 49/50 [4:44:54<00:54, 54.98s/it]

0.48 	 49 	 2923.6621904121685 	 -0.17192128722399747
0.49 	 1 	 4343.810827708472 	 -0.07619811907607454
0.49 	 2 	 4437.46407444872 	 -0.12310438190737605
0.49 	 3 	 4369.2573031290585 	 -0.08884400234065337
0.49 	 4 	 4389.399588391976 	 -0.09890628881805297
0.49 	 5 	 4387.384603258835 	 -0.09789759888238314
0.49 	 6 	 4385.917371777403 	 -0.09716340267570134
0.49 	 7 	 4386.062937774454 	 -0.097236232289867
0.49 	 8 	 4387.090364289879 	 -0.09775034330564214
0.49 	 9 	 4384.322289141506 	 -0.09636550891275997
0.49 	 10 	 4385.534777837818 	 -0.09697199471046347
0.49 	 11 	 4385.561337752093 	 -0.09698528183362765
0.49 	 12 	 4386.484413743885 	 -0.0974471188781807
0.49 	 13 	 4386.731949498265 	 -0.09757098347673043
0.49 	 14 	 4382.688761260969 	 -0.09554868484299384
0.49 	 15 	 4382.259369565231 	 -0.09533402371206856
0.49 	 16 	 4382.183135382164 	 -0.0952959149826953
0.49 	 17 	 4381.890766694215 	 -0.09514976885543724
0.49 	 18 	 4381.858659651128 	 -0.09513372012493782
0.49 

100%|████████████████████████████████████████| 50/50 [4:45:12<00:00, 342.25s/it]

0.49 	 49 	 4378.916250660286 	 -0.09366345389314845
CPU times: user 14h 8min 15s, sys: 12min 1s, total: 14h 20min 16s
Wall time: 4h 45min 12s


In [10]:
errores.sort_values(by='RMSE', ascending=True)

,Modelo,Cota,Num Paises,Acierto exacto (%),RMSE,MSE,MAE,R2
16631,BayesianRidge(),0.48,24,0.0,2912.247531,8481185.682687,2439.626717,-0.162788
16806,BayesianRidge(),0.48,49,0.0,2923.66219,8547800.603646,2463.08872,-0.171921
16687,BayesianRidge(),0.48,32,0.0,2923.66219,8547800.603646,2463.08872,-0.171921
16659,BayesianRidge(),0.48,28,0.0,2923.66219,8547800.603646,2463.08872,-0.171921
16701,BayesianRidge(),0.48,34,0.0,2923.66219,8547800.603646,2463.08872,-0.171921
...,...,...,...,...,...,...,...,...
1359,LinearRegression(),0.03,48,0.0,4140464614757.546387,17143447226059357333487616.0,732791663911.627441,-5384812799165424.0
330,LinearRegression(),0.0,48,0.0,4140464614757.546387,17143447226059357333487616.0,732791663911.627441,-5384812799165424.0
10949,LinearRegression(),0.31,46,0.0,4821641144021.44043,23248223321720388170285056.0,946809629719.167969,-7759679954263878.0
11992,LinearRegression(),0.34,48,0.0,5602381678700.734375,31386680473841653591834624.0,1165283886393.4375,-12149737697174764.0


In [11]:
errores.to_csv('errores.csv', index=False)
errores.to_excel('errores.xlsx', index=False)